In [1]:
import os
import re
import sys
import math 
import fiona
import random
import logging
import datetime
import numpy as np
import pandas as pd
import geopandas as gpd
import skimage
import skimage.draw
import pyproj
import rasterio
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_KEA.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_KEA.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_KEA.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_KEA.dll
127: The specified procedure could not be found.



## Grid by myself

In [45]:
def get_grid_idx(cell_size=64, height=2357, width=1892):
    num_h = math.ceil(height / cell_size)
    num_w = math.ceil(width / cell_size)
    num_range = np.arange(num_h * num_w).reshape(num_h, num_w)
    print(f'There are {num_h * num_w} grid groups in total.')
    grid_idx = num_range.repeat(cell_size, axis=0).repeat(cell_size, axis=1)
    print(f'before: {grid_idx.shape}')
    grid_idx = grid_idx[:height, :width]
    print(f'after: {grid_idx.shape}')
    return grid_idx

In [46]:
grid_idx = get_grid_idx()

There are 1110 grid groups in total.
before: (2368, 1920)
after: (2357, 1892)


In [47]:
1110*0.2

222.0

In [51]:
grid_idx_test = random.sample(list(np.arange(1110)), math.ceil(1110*0.2))
grid_idx_train_val = list(set(np.arange(1110)) - set(grid_idx_test))

In [53]:
pred_path = '../preds/1008-183014_rfc.tif'
band_pred, meta_pred = load_geotiff(pred_path)

In [66]:
def visualize_train_test_grid_split(meta_src, grid_idx_test, grid_idx_train_val, spatial_dict, save_path):
    # get the grid idx
    cell_size, height, width = spatial_dict['cell_size'], spatial_dict['height'], spatial_dict['width']
    grid_idx = get_grid_idx(cell_size, height, width).reshape(-1)
    # build 
    output = np.zeros_like(grid_idx)
    test_mask = [True if idx in grid_idx_test else False for idx in grid_idx]
    train_val_mask = [True if idx in grid_idx_train_val else False for idx in grid_idx]
    output[test_mask] = 1
    output[train_val_mask] = 2
    
    with rasterio.Env():
        # Write an array as a raster band to a new 8-bit file. We start with the profile of the source
        out_meta = meta_src.copy()
        out_meta.update(
            dtype=rasterio.uint8,
            count=1,
            compress='lzw')
        with rasterio.open(save_path, 'w', **out_meta) as dst:
            # reshape into (band, height, width)
            dst.write(output.reshape(1, out_meta['height'], out_meta['width']).astype(rasterio.uint8))
            dst.write_colormap(
            1, {
                0: (0, 0, 0), 
                1: (240, 72, 72),
                2: (71, 208, 240)})
            
def load_geotiff(path, window=None):
    """ Load the geotiff as a list of numpy array.
        INPUT : path (str) -> the path to the geotiff
                window (rasterio.windows.Window) -> the window to use when loading the image
        OUTPUT : band (list of numpy array) -> the different bands as float scaled to 0:1
                 meta (dictionary) -> the metadata associated with the geotiff
    """
    with rasterio.open(path) as f:
        band = [skimage.img_as_float(f.read(i+1, window=window)) for i in range(f.count)]
        meta = f.meta
        if window is not None:
            meta['height'] = window.height
            meta['width'] = window.width
            meta['transform'] = f.window_transform(window)
    return band, meta

In [67]:
spatial_dict = {'cell_size': 64, 'height': meta_pred['height'], 'width': meta_pred['width']}
visualize_train_test_grid_split(meta_pred, grid_idx_test, grid_idx_train_val, spatial_dict, './train_test_split.tiff')

There are 1110 grid groups in total.
before: (2368, 1920)
after: (2357, 1892)


In [1]:
1

1

In [2]:
class Model(object):
    def __init__(self, logger, log_time, model_name):
        self._logger = logger
        self._log_time = log_time
        self.model_name = model_name

    def _check_model_name(self, model_list):
        if self.model_name not in model_list:
            raise ValueError(f'No such model {self.model_name}. Please choose from {model_list}.')
    
    def fit_best(self):
        raise NotImplementedError

In [3]:
class ModelCropland(Model):
    def __init__(self, logger, log_time, model_name):
        super().__init__(logger, log_time, model_name)
        self._check_model_name(model_list=['svc', 'rfc', 'mlp', 'gru'])
        self.best_params = None
        self.model = None
    
    def fit_best(self, arg):
        print(f'Fit best in ModelCropSpecific with arg {arg}')

In [14]:
class ModelCropSpecific(Model):
    def __init__(self, logger, log_time, model_name, pretrained_name=None):
        super().__init__(logger, log_time, model_name)
        self._check_model_name(model_list=['ocsvm', 'pul', 'pul-w'])
        if pretrained_name is None:
            self.base_model = None
            print('Train from scratch.')
        else:
            self.base_model = pretrained_name
            print(f'Loaded pretrained model {pretrained_name}.')
        
        self.best_params = None
        self.model = None
        
    def fit_best(self, arg):
        print(f'Fit best in ModelCropSpecific with arg {arg}')
        
    def evaluate(self):
        print('Add evaluation method')

In [5]:
model_c = ModelCropland('logger', 'log_time', 'model')

ValueError: No such model model. Please choose from ['svc', 'rfc', 'mlp', 'gru'].

In [7]:
model_c = ModelCropland('logger', 'log_time', 'svc')
model_c.fit_best('xx')

Fit best in ModelCropSpecific with arg xx


In [8]:
model_cs = ModelCropland('logger', 'log_time', 'model')

ValueError: No such model model. Please choose from ['svc', 'rfc', 'mlp', 'gru'].

In [12]:
model_cs = ModelCropSpecific('logger', 'log_time', 'ocsvm')
model_cs.fit_best('yy')

Train from scratch.
Fit best in ModelCropSpecific with arg yy


In [15]:
model_cs = ModelCropSpecific('logger', 'log_time', 'ocsvm', 'pretrained_model')
model_cs.fit_best('yy')
model_cs.evaluate()

Loaded pretrained model pretrained_model.
Fit best in ModelCropSpecific with arg yy
Add evaluation method


In [16]:
import numpy as np

In [20]:
a = np.zeros((5,2))
a

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [21]:
np.concatenate([a,a], axis=0)

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [22]:
import pandas as pd

In [25]:
x = pd.DataFrame([[1,2],[2,3]], columns=list('ab'))

In [26]:
x

,a,b
0,1,2
1,2,3


In [29]:
x.a.values.shape

(2,)

In [31]:
from src.util import load_target_shp, compute_mask

ModuleNotFoundError: No module named 'src'

In [32]:
from .src.util import load_target_shp, compute_mask

ImportError: attempted relative import with no known parent package

In [33]:
from ..src.util import load_target_shp, compute_mask

ImportError: attempted relative import with no known parent package

In [37]:
import os
import re
import sys
import math
import fiona
import logging
import datetime
import numpy as np
import pandas as pd
import geopandas as gpd
import skimage
import skimage.draw
import pyproj
import rasterio
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling

def load_geotiff(path, window=None):
    """ Load the geotiff as a list of numpy array.
        INPUT : path (str) -> the path to the geotiff
                window (rasterio.windows.Window) -> the window to use when loading the image
        OUTPUT : band (list of numpy array) -> the different bands as float scaled to 0:1
                 meta (dictionary) -> the metadata associated with the geotiff
    """
    with rasterio.open(path) as f:
        band = [skimage.img_as_float(f.read(i + 1, window=window)) for i in range(f.count)]
        meta = f.meta
        if window is not None:
            meta['height'] = window.height
            meta['width'] = window.width
            meta['transform'] = f.window_transform(window)
    return band, meta

def load_target_shp(path, transform=None, proj_out=None):
    """ Load the shapefile as a list of numpy array of coordinates
        INPUT : path (str) -> the path to the shapefile
                transform (rasterio.Affine) -> the affine transformation to get the polygon in row;col format from UTM.
        OUTPUT : poly (list of np.array) -> list of polygons (as numpy.array of coordinates)
                 poly_rc (list of np.array) -> list of polygon in row-col format if a transform is given
    """
    print("Loading target shapefile...")
    with fiona.open(path) as shapefile:
        proj_in = pyproj.Proj(shapefile.crs)
        class_type = [feature['properties']['id'] for feature in shapefile]
        features = [feature["geometry"] for feature in shapefile]
    # re-project polygons if necessary
    if proj_out is None or proj_in == proj_out:
        poly = [np.array([(coord[0], coord[1]) for coord in features[i]['coordinates'][0]]) for i in
                range(len(features))]
        print('No re-projection!')
    else:
        poly = [np.array(
            [pyproj.transform(proj_in, proj_out, coord[0], coord[1]) for coord in features[i]['coordinates'][0]]) for i
            in range(len(features))]
        print(f'Re-project from {proj_in} to {proj_out}')

    poly_rc = None
    # transform in row-col if a transform is given
    if transform is not None:
        poly_rc = [np.array([rasterio.transform.rowcol(transform, coord[0], coord[1])[::-1] for coord in p]) for p in
                   poly]
    print('Loaded target shape files.')

    return poly, poly_rc, class_type


def compute_mask(polygon_list, meta, val_list):
    """ Get mask of class of a polygon list
        INPUT : polygon_list (list od polygon in coordinates (x, y)) -> the polygons in row;col format
                meta -> the image width and height
                val_list(list of int) -> the class associated with each polygon
        OUTPUT : img (np.array 2D) -> the mask in which the pixel value reflect it's class (zero being the absence of class)
    """
    img = np.zeros((meta['height'], meta['width']), dtype=np.uint8)  # skimage : row,col --> h,w
    i = 0
    for polygon, val in zip(polygon_list, val_list):
        rr, cc = skimage.draw.polygon(polygon[:, 1], polygon[:, 0], img.shape)
        img[rr, cc] = val
        i += 1
    print("Added targets' mask.")
    return img

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_KEA.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_KEA.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_KEA.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_KEA.dll
127: The specified procedure could not be found.



In [48]:
img_dir = 'N:/dataorg-datasets/MLsatellite/sentinel2_images/images_danya/geotiff/'
eg = img_dir + 'L2A_T43SFR_A026271_20200703T053446.tiff'
band, meta = load_geotiff(eg)

In [41]:
def count_classes(logger, y):
    tot_num = len(y)
    for i in np.unique(y):
        y_i = y[y == i]
        msg = f'  label = {i}, pixel number = {y_i.shape[0]}, percentage = {round(len(y_i) / tot_num * 100, 2)}%'
        if logger is None:
            print(msg)
        else:
            logger.info(msg)

In [57]:
train_polygons, train_rc_polygons, train_class_list = \
    load_target_shp('../data/test/test.shp',
                    transform=meta['transform'],
                    proj_out=pyproj.Proj(meta['crs']))
train_mask = compute_mask(train_rc_polygons, meta, train_class_list)

Loading target shapefile...


C:\Users\lida\Anaconda3\envs\ch\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\lida\AppData\Local\Temp/ipykernel_13704/4014290108.py:53: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  [pyproj.transform(proj_in, proj_out, coord[0], coord[1]) for coord in features[i]['coordinates'][0]]) for i


Re-project from proj=longlat datum=WGS84 no_defs ellps=WGS84 towgs84=0,0,0 to proj=utm zone=43 datum=WGS84 units=m no_defs ellps=WGS84 towgs84=0,0,0
Loaded target shape files.
Added targets' mask.


In [58]:
test_labeled = train_mask.reshape(-1)
test_labeled = test_labeled[test_labeled!=0]

In [59]:
count_classes(None, test_labeled)

  label = 2, pixel number = 7318, percentage = 47.78%
  label = 3, pixel number = 7999, percentage = 52.22%


## Spacv

In [61]:
import spacv
import geopandas as gpd
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

df = gpd.read_file('data/baltim.geojson')

XYs = df['geometry']
X = df[['NROOM', 'BMENT', 'NBATH', 'PRICE', 'LOTSZ', 'SQFT']]
y = df['PATIO']

# Build fold indices as a generator
skcv = spacv.SKCV(n_splits=4, buffer_radius=10).split(XYs)

svc = SVC()

cross_val_score(svc,       # Model 
                X,         # Features
                y,         # Labels
                cv = skcv) # Fold indices

C:\Users\lida\Anaconda3\envs\ch\lib\site-packages\sklearn\cluster\_kmeans.py:1045: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 1024 or by setting the environment variable OMP_NUM_THREADS=1
  warnings.warn(
C:\Users\lida\Anaconda3\envs\ch\lib\site-packages\spacv\base_classes.py:69: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  deadzone_points = gpd.sjoin(candidate_deadzone, geometry_buffer)
C:\Users\lida\Anaconda3\envs\ch\lib\site-packages\spacv\base_classes.py:69: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  dea

array([0.68627451, 0.90666667, 0.91489362, 0.84210526])